# **나만의 모델 만들기**(StatsModels)

## 0. 필요한 라이브러리 및 보스턴 데이터 불러오기

In [118]:
# 데이터 전처리 패키지
import numpy as np
import pandas as pd

# 기계학습 모델 구축 및 평가 패키지
import scipy as sp
import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 데이터 시각화 패키지
import seaborn as sns
import matplotlib.pyplot as plt

boston = pd.read_csv("house_price.csv") #보스턴 데이터 불러오기

데이터 구조  
- 데이터: 1978년 보스턴 주택 가격  
- 관측치 개수: 506개
- 변수 개수: 설명변수 13개 / 반응변수 1개

독립 변수(원인: 예측값을 설명할 수 있는 변수)      
- CRIM: 범죄율  
- INDUS: 비소매상업지역 면적 비율  
- NOX: 일산화질소 농도  
- RM: 주택당 방 수  
- LSTAT: 인구 중 하위 계층 비율  
- B: 인구 중 흑인 비율  
- PTRATIO: 학생/교사 비율  
- ZN: 25,000 평방피트를 초과 거주지역 비율  
- CHAS: 찰스강의 경계에 위치한 경우는 1, 아니면 0  
- AGE: 1940년 이전에 건축된 주택의 비율  
- RAD: 방사형 고속도로까지의 거리  
- DIS: 직업센터의 거리  
- TAX: 재산세율  

종속 변수(결과: 예측하고자 하는 값)
- MEDV: 주택가격


## **회귀 분석 프로세스**

**사전 검증 → 모델 생성 및 모델 fit → 모델 평가 → 모델 성능 개선** <Br/><Br/>

## 1. 사전검증: EDA 및 전처리 과정
### 1-1. 기본 정보 확인, 결측치 및 이상치 처리

In [119]:
#데이터 기본정보 확인
boston


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CAT.MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4,0
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6,0
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9,0
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0,0


In [120]:
boston.info() #데이터 타입 및 결측치 확인
boston.describe() #기본적인 통계값 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CRIM      506 non-null    float64
 1   ZN        506 non-null    float64
 2   INDUS     506 non-null    float64
 3   CHAS      506 non-null    int64  
 4   NOX       506 non-null    float64
 5   RM        506 non-null    float64
 6   AGE       506 non-null    float64
 7   DIS       506 non-null    float64
 8   RAD       506 non-null    int64  
 9   TAX       506 non-null    int64  
 10  PTRATIO   506 non-null    float64
 11  B         506 non-null    float64
 12  LSTAT     506 non-null    float64
 13  MEDV      506 non-null    float64
 14  CAT.MEDV  506 non-null    int64  
dtypes: float64(11), int64(4)
memory usage: 59.4 KB


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CAT.MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806,0.166008
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104,0.372456
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000,0.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000,0.000000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000,0.000000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000,0.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000,1.000000


**결측치 없음, 이상치 존재하지만 오류가 아닌 실제 관측된 값이므로 보존하고 추후 조정 또는 해석**

### X: 독립변수('MEDV' 제외 컬럼), y: 종속변수('MEDV' 컬럼)

X, y를 데이터프레임 형태로 변환


In [121]:
X = boston.drop(labels='MEDV',axis=1)
y = boston["MEDV"]

### 1-2. 변수간 상관관계 분석
**먼저 MEDV 파생 컬럼 CAT.MEDV을 drop 해주고**

In [122]:
X.drop('CAT.MEDV',axis=1,inplace=True)

**VIF 지수를 구해 다중공선성이 높은 변수를 차례로 제거한다**

In [123]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
def vif(X):
    vif_data = pd.DataFrame()
    vif_data["Features"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(vif_data)
vif(X)

   Features        VIF
0      CRIM   2.100373
1        ZN   2.844013
2     INDUS  14.485758
3      CHAS   1.152952
4       NOX  73.894947
5        RM  77.948283
6       AGE  21.386850
7       DIS  14.699652
8       RAD  15.167725
9       TAX  61.227274
10  PTRATIO  85.029547
11        B  20.104943
12    LSTAT  11.102025


**VIF가 모두 15 이하가 되도록 변수들을 하나씩 drop 시키자**

In [124]:
vif(X.drop('PTRATIO',axis=1))

   Features        VIF
0      CRIM   2.099345
1        ZN   2.451624
2     INDUS  14.275283
3      CHAS   1.142167
4       NOX  73.894171
5        RM  60.598846
6       AGE  21.361234
7       DIS  12.221605
8       RAD  15.159162
9       TAX  59.301541
10        B  18.614751
11    LSTAT  10.138324


In [125]:
vif(X.drop(['PTRATIO','RM'],axis=1))

   Features        VIF
0      CRIM   2.095455
1        ZN   2.433975
2     INDUS  13.958978
3      CHAS   1.141165
4       NOX  50.491344
5       AGE  19.961762
6       DIS   9.573301
7       RAD  15.102917
8       TAX  58.686042
9         B  16.588839
10    LSTAT   8.480702


In [126]:
vif(X.drop(['PTRATIO','RM','TAX'],axis=1))

  Features        VIF
0     CRIM   2.095430
1       ZN   2.349024
2    INDUS  11.010840
3     CHAS   1.119830
4      NOX  46.856834
5      AGE  19.899777
6      DIS   9.086148
7      RAD   5.167844
8        B  16.435073
9    LSTAT   8.468935


In [127]:
vif(X.drop(['PTRATIO','RM','TAX','NOX'],axis=1))

  Features        VIF
0     CRIM   2.095367
1       ZN   2.334763
2    INDUS   9.016142
3     CHAS   1.116229
4      AGE  14.000758
5      DIS   8.447694
6      RAD   4.771767
7        B  13.537020
8    LSTAT   8.358925


In [128]:
vif(X.drop(['RM','PTRATIO','NOX','TAX', 'AGE'],axis=1))

  Features        VIF
0     CRIM   2.095211
1       ZN   2.313889
2    INDUS   8.205465
3     CHAS   1.106350
4      DIS   8.209371
5      RAD   4.689836
6        B  10.074224
7    LSTAT   6.856889


In [129]:
#inplace=True로 변수 제거
X.drop(['RM','PTRATIO','NOX','TAX', 'AGE'],axis=1, inplace=True) 

In [130]:
#전처리 후 데이터 확인
X.head()

,CRIM,ZN,INDUS,CHAS,DIS,RAD,B,LSTAT
0,0.00632,18.0,2.31,0,4.0900,1,396.90,4.98
1,0.02731,0.0,7.07,0,4.9671,2,396.90,9.14
2,0.02729,0.0,7.07,0,4.9671,2,392.83,4.03
3,0.03237,0.0,2.18,0,6.0622,3,394.63,2.94
4,0.06905,0.0,2.18,0,6.0622,3,396.90,5.33


**마지막으로 변수들을 Train set(20%)과 Test set(80%)으로 나누어준다.**

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

## 2. 모델 생성 및 모델 fit
**모델에 상수항 추가 (절편 포함)**

In [132]:
X_train= sm.add_constant(X_train)

**이제 train 데이터를 이용해 회귀모델을 만들어보자**

In [133]:
model=sm.OLS(y_train, X_train) # model에 sm.OLS를 저장
model_trained=model.fit() # 모델을 fit시켜 model_trained변수에 저장

## 3. 모델 평가
**테스트 데이터에도 상수항 추가 후 예측**

In [134]:
X_test= sm.add_constant(X_test)
y_pred_sml = model_trained.predict(X_test) 
y_pred_sml

210    21.808564
24     17.814625
36     24.847267
439    14.483329
161    31.792370
         ...    
8       3.874279
317    18.383510
390    19.630666
482    25.811970
50     18.584977
Length: 102, dtype: float64

In [136]:
print(model_trained.summary())       

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     96.92
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.66e-88
Time:                        23:25:19   Log-Likelihood:                -1261.9
No. Observations:                 404   AIC:                             2542.
Df Residuals:                     395   BIC:                             2578.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         40.4240      1.969     20.533      0.0

### T검정 진행
- 귀무가설: 회귀 계수가 0이다.

P-value가 0.05를 넘지 않는 변수들에 대해 귀무가설을 기각할 수 있다.

In [135]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error          #scikit-learn으로 분석

test_mse_sml = mean_squared_error(y_test, y_pred_sml) 
test_mae_sml = mean_absolute_error(y_test, y_pred_sml)
test_r2_sml = r2_score(y_test, y_pred_sml) 

In [137]:
print(test_mse_sml)
print(test_mae_sml)
print(test_r2_sml) #결정계수

32.71687125128576
4.183405347929009
0.48780895960676196


**회귀모델이 test데이터에 대해 49퍼센트 정도의 설명력을 보인다**